In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
#import dictionaries from my_tools to automize plotting
from my_tools import plot_dic
from cmap import ncl_colormap
from mpl_toolkits.basemap import Basemap, addcyclic
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import string
# plt.rcParams.update({'font.size': 12})
%matplotlib qt

Northern Hemisphere ERA5RF in JJA and DJF

In [19]:
#set path
path_GFS_mean = '/media/onno/Algemeen/Thesis/GFS_mean_season/'
path_Era_clim = '/media/onno/Algemeen/Thesis/ERA_climatology/'
path_to_save = '/media/onno/Volume/Main_Figures/'
#List all months to plot
seasonz = ['MAM','SON']
#list all variables
variable = 'envelope'
#set climatology plot ranges
vminmax_clim = [(8,32)]
#set error plot ranges
vminmax_errors = [(-1.5,1.5)]
#I only chose to plot forecast errors of day 2,5, and 8 for convenience
dayz = [2,4,8] 
#Set custom colormap for climatology
my_cmap = ncl_colormap()
#create array of letters
letterz = np.array(list(string.ascii_lowercase)[:12]).reshape(3,4)
#initialize figure
fig,axz = plt.subplots(2,4,figsize=(9,5))
for y,season in enumerate(seasonz):
    #make room for colorbars
    fig.subplots_adjust(hspace=0.15,left=0.07,right=0.97,top=0.875,bottom=0.1,wspace=0.25)
#     fig.suptitle("Results {}".format(season),fontsize=16)
    #loop over y and x axis in figure
    for x in range(axz.shape[1]):
        ax = axz[y,x]
        ax.text(x=-0.15,y=1,s=letterz[y,x]+')',transform=ax.transAxes,fontweight='bold',fontsize=12,
               verticalalignment='top')
        #in first column only plot ERA5 climatologies
        if x==0:
            #open climatology file for each variable
            file = 'ERA5_{}_clim_{}_mars_1979.nc'.format(variable,season)
            ds = xr.open_dataset(path_Era_clim+file,decode_times=False).squeeze()
            #remove data near the North Pole
            ds = ds.sel(lat=slice(85,0))
            #add cyclic point for contourf function to work properly
            ds_cyclic, lon_cyclic = addcyclic(ds[plot_dic[variable]['data_variable']].values, ds.lon)
            lon,lat = np.meshgrid(lon_cyclic,ds.lat)
            m = Basemap(projection='ortho',lon_0=0,lat_0=90,resolution='c',ax=ax)
            im = m.contourf(lon,lat,ds_cyclic,latlon='true', extend='both', cmap=my_cmap,
               vmin=vminmax_clim[0][0],vmax=vminmax_clim[0][1],levels=np.arange(vminmax_clim[0][0],vminmax_clim[0][1]+0.5,0.5))
            #Define layout of geoaxes
            m.drawcoastlines()
            m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
            m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1],fontsize=6)
            #set title above column
            if y==0:
                ax.text(x=0.5,y=1.15,s='ERA5 Climatology',transform=ax.transAxes,horizontalalignment='center',fontsize=12)
            #define label for each row
            ax.text(x=-0.25,y=0.5,s="E (m/s) {}".format(season),transform=ax.transAxes,verticalalignment='center',fontsize=12,rotation=90)
            #define new axis for colorbar just underneath every ax of first column
            if y==1:
                cbax = fig.add_axes([ax.get_position().x0-0.04,ax.get_position().y0-0.04,ax.get_position().x1-ax.get_position().x0+0.08,0.015])
        
                fig.colorbar(im, cax=cbax,ticks=np.linspace(vminmax_clim[0][0],vminmax_clim[0][1],5),orientation='horizontal')
        #For the other columns plot the day 2,5,8 error data for each variable
        else:
            #open file

            if (dayz[x-1]>4)&(variable=='phasespeed'):
                file = 'ERA5RF_mean_error_{}_{}_day_{:02d}_1979.nc'.format(variable,season,0)
                ds = xr.open_dataset(path_GFS_mean+file,decode_times=False)
                ds = ds.sel(lat=slice(85,0))
            else:
                file = 'ERA5RF_mean_error_{}_{}_day_{:02d}_1979.nc'.format(variable,season,dayz[x-1])
                ds = xr.open_dataset(path_GFS_mean+file,decode_times=False)
                ds = ds.sel(lat=slice(85,0))                    
            #remove data near the North Pole

            #Add cyclic point for contourf function to work
            ds_cyclic, lon_cyclic = addcyclic(ds[plot_dic[variable]['data_variable']].values, ds.lon)
            lon,lat = np.meshgrid(lon_cyclic,ds.lat)
            m = Basemap(projection='ortho',lon_0=0,lat_0=90,resolution='c',ax=ax)
            im = m.contourf(lon,lat,ds_cyclic,latlon='true', extend='both', cmap=plt.cm.get_cmap('bwr'),
               vmin=vminmax_errors[0][0],vmax=vminmax_errors[0][1],levels=np.linspace(vminmax_errors[0][0],vminmax_errors[0][1],49))
            #define layout for each geoaxes
            m.drawcoastlines()
            m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
            m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1],fontsize=6)    
            #For top row set title which applies to column which day it is
            if y==0:
                ax.text(x=0.5,y=1.15,s='ERA5RF Day {}\nForecast Error'.format(dayz[x-1]),transform=ax.transAxes,horizontalalignment='center',fontsize=12)
            #For every third colum set colorbar for error data just beneath every ax
            if (x==2)&(y==1):
                cbax = fig.add_axes([ax.get_position().x0-0.04,ax.get_position().y0-0.04,ax.get_position().x1-ax.get_position().x0+0.08,0.015])
                fig.colorbar(im, cax=cbax,ticks=np.linspace(vminmax_errors[0][0],vminmax_errors[0][1],5),orientation='horizontal')
fig.savefig(path_to_save+'E_biases_NH_ERA5RF_1979_MAM_SON.png'.format(season),dpi=200)  
# plt.close(fig)

